In [2]:
import sys
sys.path.append('..')

from ml.dataset.corpus import *
from ml.dataset.mappers_preprocess import *
from ml.dataset.mappers_simplify import *
from ml.dataset.pipeline import *
from IPython.core.display import HTML, display

import matplotlib.pyplot as plt
import pandas as pd
import pychord
import music21
music21.environment.set('musescoreDirectPNGPath', '/usr/bin/musescore')

import matplotlib.pyplot as plt
import numpy as np
import pickle

y = np.load("y_simple_my.npy")
X = (np.load("X_simple_my.npy"))

In [3]:
print(X.shape, y.shape)

(246, 56) (246,)


In [4]:
els, counts = np.unique(y, return_counts=True)
# plt.scatter(range(len(els)), counts)
# plt.show()
# print(els, counts)
rare = els[counts < 10]

isin = np.logical_not(np.isin(y, rare))
print(X.shape)
X = X[isin, :]
print(X.shape)
y = y[isin]

els, counts = np.unique(y, return_counts=True)
# plt.scatter(range(len(els)), counts)
# plt.show()
print(els)
print(len(els))

feat = X.shape[1]
print("Features:", feat)
tst = np.array([1])
print(tst.dtype, X.dtype)
if X.dtype == tst.dtype:
    num_features = np.zeros(feat, dtype='bool')
else:
    num_features = np.array(['0' <= X[0, i][0] <= '9' for i in range(feat)])
cat_features = np.logical_not(num_features)
print(num_features)
print(cat_features)

(246, 56)
(171, 56)
['' 'A3' 'A4' 'C3' 'C4' 'D4' 'E4' 'E5' 'G4']
9
Features: 56
int64 <U32
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False]
[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True]


In [5]:
# X_cat = X[:,8:16]
# X_real = np.int_(np.hstack([X[:,:8], X[:,16:]]))

X_cat = X[:, cat_features]
X_real = np.int_(X[:, num_features])
print(X_cat[0, :])
print(X_real[0, :])

['E4' 'E4' 'E4' 'E4' 'E4' 'E4' 'E4' 'E4' 'E4' 'E4' 'E4' 'E4' 'A4' 'A4'
 'A4' 'A4' 'D4' 'D4' 'D4' 'D4' 'G4' 'G4' 'G4' 'G4' 'F4' 'F4' 'F4' 'F4']
[51 57 66 59 52 55 64 59 52 56 64 59 52 57 64 60 50 57 66 62 50 55 67 59
 53 57 64 60]


In [6]:
X_cat

array([['E4', 'E4', 'E4', ..., 'F4', 'F4', 'F4'],
       ['A2', '', '', ..., '', '', 'E2'],
       ['A2', '', '', ..., '', '', 'E2'],
       ...,
       ['', 'D4', '', ..., 'A4', '', 'A4'],
       ['G3', 'G3', 'B2', ..., 'G3', 'D3', 'D3'],
       ['G2', '', '', ..., 'F3', 'F3', 'F3']], dtype='<U32')

In [7]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
enc.fit(X_cat.reshape(X_cat.size))
for i in range(sum(cat_features)):
    print(enc.classes_)
for i in range(sum(cat_features)):
    X_cat[:, i] = enc.transform(X_cat[:, i])

['' 'A2' 'A3' 'A4' 'A5' 'B2' 'B3' 'B4' 'Bb3' 'C#4' 'C#5' 'C3' 'C4' 'C5'
 'D3' 'D4' 'D5' 'E2' 'E3' 'E4' 'E5' 'Eb3' 'Eb4' 'Eb5' 'F#4' 'F2' 'F3' 'F4'
 'G#3' 'G#4' 'G2' 'G3' 'G4' 'G5']
['' 'A2' 'A3' 'A4' 'A5' 'B2' 'B3' 'B4' 'Bb3' 'C#4' 'C#5' 'C3' 'C4' 'C5'
 'D3' 'D4' 'D5' 'E2' 'E3' 'E4' 'E5' 'Eb3' 'Eb4' 'Eb5' 'F#4' 'F2' 'F3' 'F4'
 'G#3' 'G#4' 'G2' 'G3' 'G4' 'G5']
['' 'A2' 'A3' 'A4' 'A5' 'B2' 'B3' 'B4' 'Bb3' 'C#4' 'C#5' 'C3' 'C4' 'C5'
 'D3' 'D4' 'D5' 'E2' 'E3' 'E4' 'E5' 'Eb3' 'Eb4' 'Eb5' 'F#4' 'F2' 'F3' 'F4'
 'G#3' 'G#4' 'G2' 'G3' 'G4' 'G5']
['' 'A2' 'A3' 'A4' 'A5' 'B2' 'B3' 'B4' 'Bb3' 'C#4' 'C#5' 'C3' 'C4' 'C5'
 'D3' 'D4' 'D5' 'E2' 'E3' 'E4' 'E5' 'Eb3' 'Eb4' 'Eb5' 'F#4' 'F2' 'F3' 'F4'
 'G#3' 'G#4' 'G2' 'G3' 'G4' 'G5']
['' 'A2' 'A3' 'A4' 'A5' 'B2' 'B3' 'B4' 'Bb3' 'C#4' 'C#5' 'C3' 'C4' 'C5'
 'D3' 'D4' 'D5' 'E2' 'E3' 'E4' 'E5' 'Eb3' 'Eb4' 'Eb5' 'F#4' 'F2' 'F3' 'F4'
 'G#3' 'G#4' 'G2' 'G3' 'G4' 'G5']
['' 'A2' 'A3' 'A4' 'A5' 'B2' 'B3' 'B4' 'Bb3' 'C#4' 'C#5' 'C3' 'C4' 'C5'
 'D3' 'D4' 'D5' 'E2' 'E

In [8]:
X_cat

array([['19', '19', '19', ..., '27', '27', '27'],
       ['1', '0', '0', ..., '0', '0', '17'],
       ['1', '0', '0', ..., '0', '0', '17'],
       ...,
       ['0', '15', '0', ..., '3', '0', '3'],
       ['31', '31', '5', ..., '31', '14', '14'],
       ['30', '0', '0', ..., '26', '26', '26']], dtype='<U32')

In [9]:
X = np.hstack([np.int_(X_cat), X_real])

In [10]:
X

array([[19, 19, 19, ..., 57, 64, 60],
       [ 1,  0,  0, ...,  0,  0, 40],
       [ 1,  0,  0, ..., 33, 33, 40],
       ...,
       [ 0, 15,  0, ..., 33, 40, 40],
       [31, 31,  5, ..., 35, 38, 38],
       [30,  0,  0, ..., 33, 36, 33]])

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from nottingham_test import custom_scorers
from sklearn.metrics import make_scorer

clf = RandomForestClassifier(n_estimators=20)
clf.fit(X_train, y_train)

print(X_test)

#abs_dist_score = make_scorer(custom_scorers.chords_dist_error)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))
#print(abs_dist_score(clf, X_test, y_test))

with open('rf_nottingham.pkl', 'wb') as fid:
    pickle.dump(clf, fid)
print(y_test[10], X_test[10, :], clf.predict(X_test)[10])

[[ 0 15  0 ... 38 33 33]
 [ 0  2  0 ... 31 38 38]
 [ 0 32  0 ... 43 40 40]
 ...
 [ 5  0  0 ... 66 62 55]
 [ 0 20  0 ... 38 42 42]
 [11 11 18 ... 48 40 40]]
0.813953488372093
D4 [ 0 15  0 15  0 15  0 15  0 15  0 15  0 15  0 15  0 15  0 15  0 15  0 15
  0 15  0 15 43 43 38 38 43 43 38 38 41 41 36 36 41 41 38 38 43 43 38 38
 43 43 38 38 38 38 45 45] D4


/home/sharik/.pyenv/versions/3.6.0/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
